## Project Euler \#318

In [3]:
#318

(2**0.5+3**0.5)**2

9.898979485566358

In [4]:
(2**0.5+3**0.5)**4

97.98979485566359

In [7]:
(2**0.5+3**0.5)**8

9601.999895855035

In [13]:
(2**0.5+3**0.5)**6

969.9989690710697

In [9]:
import sympy
9601**0.5

97.98469268207153

In [10]:
98**2

9604

In [11]:
10, 10**2-2, 98**2-2

(10, 98, 9602)

In [12]:
(10**2-2)**2-2

9602

In [15]:
98*10-10

970

In [17]:
970*10-98

9602

近似整数を返す関数を作ってみる。

In [68]:
from functools import lru_cache

@lru_cache(maxsize=32)
def approx(n):
    if n==1:
        return 10
    elif n==0:
        return 2
    else:
        return approx(n-1)*10 - approx(n-2)

approx(4)

9602

実数を返す関数を作ってみる。

In [24]:
def f(n):
    return (2**0.5+3**0.5)**(2*n)

f(4)

9601.999895855035

In [ ]:
解析的な値を返す関数を作ってみる。

In [29]:
def sf(n):
    return (sympy.sqrt(2)+sympy.sqrt(3))**(2*n)

sympy.expand(sf(4))
    

1960*sqrt(6) + 4801

この値の整数部分は、上のapprox(4)のちょうど半分。ということは、$1960*\sqrt 6$は極めて$9602-4801=4801$に近いことになる。つまり$\sqrt 6\sim{4801\over 1960}$。こういう、極めて無理数に近い分数をさがすには、連分数展開が便利。

In [57]:
4801-1960*6**0.5

0.00010414497137389844

In [51]:
sympy.expand(sf(5))

19402*sqrt(6) + 47525

連分数展開。

In [46]:
def seriesf(x,n):
    a = int(x)
    if n==1:
        return [int(x)]
    if a == x:
        return []
    return [a] + seriesf(1/(x-a),n-1)

# seriesf(3.1415926535,5)
seriesf(6**0.5,10)   
    

[2, 2, 4, 2, 4, 2, 4, 2, 4, 2]

2,4がくりかえされる規則性がある。

連分数を分数に戻す関数。

In [71]:
import fractions

def unseriesf(L):
    if len(L) == 1:
        return fractions.Fraction(L[0],1)
    return L[0]+1/unseriesf(L[1:])

def unseriesf2(L):
    """
    no recursion
    """
    x = fractions.Fraction(L[-1],1)
    for i in range(len(L)-2,-1,-1):
        x = L[i]+1/x
    return x

#unseriesf([3,7,15,1])
unseriesf2([2,2,4,2,4,2,4,2])

Fraction(4801, 1960)

図星。4,2の繰り返しをとちゅうで打ちきると、分数の分子分母がさっきでてきた値にぴったり一致する。

In [52]:
unseriesf([2,2,4,2,4,2,4,2,4,2])

Fraction(47525, 19402)

ということは? 厳密な$\sqrt 6$の分母倍と、分子の差が急激に0に近付く。それが$e<10^{-2011}$以下になるようなNをまずさがす。

微小な差を桁落ちなしに求める方法はどうしよう? 連分数を延長すると、それだけ正確になる。ということは連分数を1段(4,2)だけ伸ばした時に、精度がどれだけよくなるかがわかれば、それはほとんど$e$に近いはず。それはどんな分数で表されるか。


In [60]:
unseriesf([2,2]+[4,2]*(3-1))

Fraction(485, 198)

In [61]:
unseriesf([2,2]+[4,2]*(4-1))

Fraction(4801, 1960)

In [63]:
(unseriesf([2,2]+[4,2]*(3-1))-unseriesf([2,2]+[4,2]*(4-1)))*1960

Fraction(1, 99)

つまり$[2;2,4,2,4,2]$と$[2;2,4,2,4,2,4,2]$の差は${1\over 99\cdot 1960}$である。今知りたいのは$198\sqrt 6-485$で、それはおおよそ${1\over 3920}=0.0002$になる、はず。

In [64]:
198*6**0.5-485

-0.0010309289307883773

In [65]:
(198*6**0.5-485)*1960

-2.0206207043452196

経験的な数値として、$N$までの連分数展開での誤差は、$N+1$までの連分数の分母を$a$とした時,$2/a$程度である。では、$a$が2011桁になるのは$N$がいくつぐらいかをおおよそ推定する。

In [69]:
N = 0
a = 10
while a < 10**2011:
    N += 1
    a = approx(N)

print(N)

2020


In [77]:
unseriesf2([2,2]+[4,2]*(2021-1))

Fraction(6126569641221458552996768136518532821743371308074260750926200006673856036884306845349292080899719037391221580702847503096691658873414327319944781903986740078340449660205520788141088682657543420758932759806355144471791573680820084535146551718100254741856949256975184545140461122822608578522551865398289389209036591338995613731048564748693967534495539597902024654417247187954367304352705340887418318628967630467057590050681260574737294967014605566768898704877145439142658268456332162106007065025557516346761557202952444511086951819625179780921095505945759960788798864776980596486252362054317890117475408710467422723370427329583653128632829102084906585541085380124247638505162399277452694051569963186994287809919432487550971666194666023867485026416317321405251218346409389571434402365565883773546187954957293501523247204946011416517218557284397244584790271503526115793784165224697935375111352988779091195982437294209271435706591562626843677701444851226855466388191664781036672149352603819703032

In [79]:
sympy.N("2/25011615824364630393214643064443372101052949344239295938507522730808201749013895380103808132007386021451262335713340660427438731791161328588700108158332063162668589280212262885331730254249644066591942246227532229372933249646118635151353683917326421257444188347710627449541038270496583766411531262631095442151047627966806627764490391463869366625417419585366994235180471665393985682492254897715403836178323048646930111776749493436916922830670911845164534420596201733288376621597037061773077011949277154288432639701151704964532793758212977666274276161402779672194567531971141747898503608479822026461212366114186907722736447800922777383304259123230523942900796948996429493624364386347819336954178499154532048060760350177462269870807064952867726091456365331326056820890725260205065217819198916151970882555483826762632182255900939375483431887660997612252472208226988576363959989967021682017125015565366551723068915744738228034883745244122937532779173371151443061332512941654889493340424660748966617183146009651711815893116677182937767889588459022935679444746221133841106994383665855731334669138817135498456405874792591349236873244903307645388109141927352959820329701734465497629104926572177097199667458312944337188082302085439293251246815357952552019386736831605040673613532062613420171349252489794386698082862292798140541177610898552151437542825294013246721181756743475549951245281163817084200540710055811069922909384931232628731154020823999502868972994979372713801918450743530691756826334098733127631782109919509208450221833441457503198990717450685607045767382868082660173874618442801540948503988497538889027013039429547145260445367829685433615869718988409703205942625713475953124489039633150079297237111859210920068358374072151439673366709194972299563181178046236828158862773552413523056779992283545938798086795875290025367065341184858791930265702117753072919947571365693353492603234934581484508974827975531449035605546151978450400332254132186971274563705166443465983922391587781803650726891538901992739926190789002")

7.99628466247165e-2012

$1-7.99e-2$は0.92であり、9は1個。同じように、$1-8e-2012$は9が2011個並ぶ数である。つまり、$N=2020$の場合に、9が2011個並ぶ。

検算しよう。

In [84]:
N=3
v = unseriesf2([2,2]+[4,2]*(N+1-1))
v

Fraction(4801, 1960)

In [88]:
sympy.N(fractions.Fraction(2,v.denominator))

0.00102040816326531

$1-0.0010=0.998$で9が2個並ぶ。これはNが3の場合。問題ない。

再確認

In [92]:
N=2020
v = unseriesf2([2,2]+[4,2]*(N+1-1))
sympy.N(fractions.Fraction(2,v.denominator))

7.99628466247165e-2012

これでやっとp,q=2,3の場合が解けた。一般的なp,qに拡張する場合、連分数展開がうまくつかえるのかどうかが不明。道程は遠い。